In [19]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import os

from uszipcode import SearchEngine
import geocoder

import json
import requests

from scipy.stats import sem
from pprint import pprint

In [2]:
#Import files and read in data

data = "Resources/Zip_MedianValuePerSqft_AllHomes.csv"
data_input = pd.read_csv(data, encoding="ISO-8859-1", low_memory=False)
data_input

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,200.0,200.0,201.0,...,1339,1333,1329,1321,1312,1298,1280,1275,1271,1257
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,156.0,157.0,157.0,...,475,475,476,476,477,478,479,480,485,489
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,359.0,359.0,359.0,...,1630,1612,1601,1589,1577,1564,1544,1524,1502,1478
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,67.0,68.0,68.0,...,113,113,113,113,114,114,114,114,114,114
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,199.0,200.0,201.0,...,523,524,523,523,525,526,527,529,532,532
5,91940,77449,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,6,49.0,49.0,49.0,...,92,92,93,93,93,93,94,95,95,94
6,93144,79936,El Paso,TX,El Paso,El Paso County,7,54.0,54.0,55.0,...,88,89,89,89,89,90,90,91,91,91
7,91733,77084,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,8,49.0,50.0,49.0,...,87,87,88,89,89,90,91,91,91,90
8,61616,10002,New York,NY,New York-Newark-Jersey City,New York County,9,270.0,270.0,270.0,...,1427,1427,1424,1415,1403,1387,1371,1369,1363,1346
9,61807,10467,New York,NY,New York-Newark-Jersey City,Bronx County,10,86.0,87.0,87.0,...,263,265,267,268,269,272,274,277,281,284


In [8]:
# Convert to dataframe; drop unneeded columns; get information on data
data_input.columns

Index(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName',
       'SizeRank', '1996-04', '1996-05', '1996-06',
       ...
       '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
       '2019-01', '2019-02', '2019-03', '2019-04'],
      dtype='object', length=284)

In [10]:
data_input['2019-01'].describe()

count    14766.000000
mean       174.971895
std        158.624539
min         18.000000
25%         93.000000
50%        130.000000
75%        195.000000
max       2004.000000
Name: 2019-01, dtype: float64

In [11]:
df = data_input[['RegionName', 'City', 'State', 'Metro', 'CountyName',  'SizeRank', '2019-01']]

df = df.rename(columns={'RegionName': "zipcode", '2019-01': "medValuePSFt" })

df.head()

,zipcode,City,State,Metro,CountyName,SizeRank,medValuePSFt
0,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1280
1,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,479
2,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1544
3,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,114
4,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,527


In [12]:
# test for indivdual results
search = SearchEngine(simple_zipcode=True)

zipcode = search.by_zipcode(df.zipcode[0])
print (zipcode.lat, zipcode.lng)

40.8 -73.97


In [13]:
#convert column data into a callable list
zipcodes = []
for x in df.zipcode:
   zipcodes.append(x)

print(zipcodes)

[10025, 60657, 10023, 77494, 60614, 77449, 79936, 77084, 10002, 10467, 11226, 60640, 94109, 10016, 78660, 37013, 32162, 11235, 11375, 60647, 90250, 37211, 10029, 10009, 77573, 60618, 78130, 77584, 10011, 10128, 28269, 20002, 78613, 78572, 30349, 79912, 94565, 90046, 77433, 75052, 20009, 37042, 73099, 77429, 10462, 10456, 77479, 10003, 60613, 63376, 30044, 10019, 77036, 10463, 28277, 30043, 11212, 11230, 60625, 78245, 11209, 60629, 30096, 7030, 60610, 11221, 78704, 75287, 85032, 77077, 10024, 33160, 23464, 21234, 30024, 75243, 98052, 11214, 78521, 8701, 77379, 94501, 78666, 11207, 33411, 11229, 78745, 60619, 23462, 28027, 94110, 33025, 75067, 90034, 33139, 44107, 48197, 2169, 11377, 66062, 34787, 44035, 95630, 90044, 90805, 80134, 90650, 85710, 92683, 10314, 11211, 11234, 85281, 11355, 30281, 85225, 11215, 60617, 37075, 94558, 90026, 87114, 32210, 43081, 91910, 6010, 17603, 78240, 89108, 93722, 87111, 85308, 79938, 76063, 77459, 48180, 94533, 33908, 97229, 91709, 75056, 76028, 78577, 44

In [16]:
#insert lat and long values for each zipcode (values needed for use with leaflet.js)

lat = []
long = []

for z in zipcodes:
    search = SearchEngine(simple_zipcode=True)
    results = search.by_zipcode(z)
    lat.append(results.lat)
    long.append(results.lng)
   
df["lat"] = lat
df["lng"] = long
df.head()

,zipcode,City,State,Metro,CountyName,SizeRank,medValuePSFt,lat,lng
0,10025,New York,NY,New York-Newark-Jersey City,New York County,1,1280,40.80,-73.97
1,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,479,41.94,-87.66
2,10023,New York,NY,New York-Newark-Jersey City,New York County,3,1544,40.78,-73.98
3,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,114,29.74,-95.84
4,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,527,41.92,-87.65


In [23]:
# export results to json
df.to_json("Resources/zillow.json")

In [24]:
#call on json file to verify export

filepath = os.path.join("Resources/zilllow.json")
with open(filepath) as jsonfile:
    zillow = json.load(jsonfile)

FileNotFoundError: [Errno 2] No such file or directory: 'Resources/zilllow.json'

In [ ]:
# create map object
var myMap = L.map("map", {
  center: [40.7128, -74.0059],
  zoom: 11
});

In [ ]:
# create base layers


In [ ]:
# create overlay object

var overlayMapZillows = {
  "Price per SQ ft": states,
  "C": cities
};